In [1]:
# Imports
import numpy as np
import pandas as pd

## Data Import

In [ ]:
# Importing the data import modules
%run ./Module_DataImport.ipynb

[Concentration_byID, file_size] = ieqImport(deployment_info,bids,bbs,'pms5003',['time','PM1','PM2.5','PM10','STD1','STD2.5','STD10'],[0,1,2,3,4,5,6])